In [17]:
from flyvision import NetworkView
from flyvision.connectome import ReceptiveFields
from flyvision.utils.activity_utils import LayerActivity, SourceCurrentView

In [2]:
# checkpoint
ensemble_and_network_id = "0000/000"  # type: str
task_name = "flow" # type: str
checkpoint='best'
validation_subdir='validation'
loss_file_name='loss'

# configure recording
dt = 1 / 200  # type: float
# cell types to store responses and currents for
target_cell_types = ["T4a", "T4b", "T4c", "T4d", "T5a", "T5b", "T5c", "T5d", "TmY3"]



In [ ]:
network_view = NetworkView(
    f"{task_name}/{ensemble_and_network_id}",
    checkpoint=checkpoint,
    validation_subdir=validation_subdir,
    loss_file_name=loss_file_name,
)


# store
subdir = f"movingedge_responses/{network_view.checkpoints.current_chkpt_key}/currents"

In [ ]:
network = network_view.init_network()

In [23]:
def main(dataset):
    print("running experiment")

    del network_view.dir[subdir]
    network_view.dir[subdir].path.mkdir(parents=True, exist_ok=True)
    network_view.dir[subdir].config = dataset.config
    edges = network.connectome.edges.to_df()

    # to store the responses and currents in a structured way
    activity_indexer = LayerActivity(None, network_view.connectome, keepref=True)
    source_current_indexer = {
        target_type: SourceCurrentView(ReceptiveFields(target_type, edges), None)
        for target_type in edges.target_type.unique()
    }

    activities = []
    currents = []
    for _, activity, current in network_view.network.current_response(
        dataset,
        dt,
        indices=None,
        t_pre=2,
        t_fade_in=0,
    ):
        # implementing computation and writing here together to save some runtime memory

        # add an empty sample dimension
        activity_indexer.update(activity)
        for target_type in target_cell_types:
            network_view.dir[subdir][target_type].extend(
                "activity_central", [activity_indexer.central[target_type]]
            )
            for source_type in source_current_indexer[target_type].source_types:
                source_current_indexer[target_type].update(current)
                network_view.dir[subdir][target_type].extend(
                    source_type, [source_current_indexer[target_type][source_type]]
                )

# run moving edge stimulus

In [25]:
from flyvision.datasets.moving_bar import MovingEdge

In [27]:
dataset = MovingEdge(
        widths=[80],
        offsets=[-10, 11],
        intensities=[0, 1],
        speeds=[19],
        height=80,
        bar_loc_horizontal=0.0,
        shuffle_offsets=False,
        post_pad_mode="continue",
        t_pre=1.0,
        t_post=1.0,
        dt=1 / 200,
        angles=[0, 45, 90, 180, 225, 270],
)

In [ ]:
main(dataset)